In [93]:
"""
Small script to make for automated test dict, which has all relevant literature
values for the given tellurium model
"""
import tellurium as te
import pandas as pd
import numpy as np
index = []
inde = 0

def metabolomics_test_dict(model, metabolites, timepoints=[]):
    """
    Function to create dict of metabolomics data from literature values
    -----------
    Parameters
    -----------
    model: te.model class
        tellurium model class, used to get species names of the model
    metabolites: pandas.DataFrame
        DataFrame of literature values except Alex Maier
    """
    # Factor to change DataFrame unit[partical number/cell] to [nmol/10^9 cells]
    # alex_factor = 602214.076  # avo/(10^9 cells and 10^9 nmol/mol
    # Translation dict keys=model_names, values=DataFrame_name
    long_name = {
        'Choline': 'avg Cho',
        'L_Serine': 'avg Ser',
        'Phosphatidylethanolamine': 'avg PE',
        'Phosphatidylcholine': 'avg PC'}
    value_dict = {
        'Choline': ['Choline_vo', 'Choline_t14'],
        'L_Serine': ['Serine_t14', 'Serine_vo'],
        'Phosphatidylethanolamine': ['PE_t09', 'PE_t14'],
        'Phosphatidylcholine': ['PC_t14', 'PC_t09']
    }
    # find matching values between dataframe and model
    intersection = list(set(metabolites.index) & set(model.getFloatingSpeciesIds()))
    global inde
    global index
    work_df = metabolites.copy()
    model_names = intersection + [a for a in model.getFloatingSpeciesIds()
                                  if a in long_name]
    new_dict = {}
    stage_dict = {"uninfected": ['uRBC', 'Std.'],        # uninfected
                  "ring": None,                  # ring
                  "troph": ['Parasite', 'Std..1'],  # trophozoite
                  "schizont": None}                  # schizont
    net = []
    for row in model_names:
        key = row
        if row in value_dict.keys():
            key = value_dict[row]

        new_dict[row] = {'Values': [], 'std': []}
        for keys in timepoints:

            if stage_dict[keys] is None:
                new_dict[row]['Values'].append([np.nan])
                new_dict[row]['std'].append([np.nan])
                net.append(np.nan)
                
                index.append(inde)
                inde+=1
            else:
                values = work_df.loc[key, stage_dict[keys][0]].astype('float').tolist()
                stds = work_df.loc[key, stage_dict[keys][1]].tolist()
                # test whether if values is float, if true make a list
                if type(values) == float:
                    values = [values]
                    stds = [stds]

                new_dict[row]['Values'].append(values)
                new_dict[row]['std'].append(stds)
                net.extend(values)
                
                index.extend([inde]*len(values))
                inde+=1
                
        new_dict[row]['Values'] = np.asarray(new_dict[row]['Values'])
        
        new_dict[row]['std'] = np.asarray(new_dict[row]['std'])
    # parasite_vol = np.array([4.00**-1 * 1e-3, 26.7**-1 * 1e-3, 26.9**-1 * 1e-3])
    # extract the avs and give proper names

    return new_dict, net


def mke_test_dict(model, timepoints=["uninfected", "ring", "troph", "schizont"]):
    """
    Function to create test_dict for objective function
    -----------
    Parameters
    -----------
    model: te.model class
        tellurium model class, used to get species names of the model
    maierframe: pd.DataFrame
        DataFrame of experimental data Alex Maier
    metabolites: pandas.DataFrame
        DataFrame of literature values except Alex Maier
    """
    maierframe = pd.read_csv("Datasets/maierframe_muMolar.tsv",
                             sep="\t", index_col=0)  # maier dataset
    metabolites = pd.read_csv("Datasets/metabolites_muMolar.tsv",
                       sep="\t", index_col=0)

    long_name_to_maier = {
        '1,2-Diacyl-sn-glycerol': 'DG',
        'DAG': 'DG',
        'Phosphatidylserine_mem': 'PS',
        'Phosphatidylethanolamine_mem': 'PE',
        'Phosphatidylcholine_mem': 'PC'}

    stage_dict = {"uninfected": ['RBC1', 'RBC2', 'RBC3'],
                  "ring": ['Ring 1', 'Ring 2', 'Ring 3'],
                  "troph": ['Trophozoite 1', 'Trophozoite 2', 'Trophozoite 3'],
                  "schizont": ['Schizont 1', 'Schizont 2', 'Schizont 3']}

    # now we make datadf
    work_df = maierframe.copy()
    work_df = work_df.loc[[long_name_to_maier[a]
                           for a in model.getFloatingSpeciesIds()
                           if a in long_name_to_maier]]
   
    # parasite_vol = np.array([4.00**-1 * 1e-3, 26.7**-1 * 1e-3, 26.9**-1 * 1e-3])
    # extract the avs and give proper names
    new_dict = {}
    maier_to_long_name = {v: k for k, v in long_name_to_maier.items()}
    names = []
    all_vals = []
    global index
    global inde
    for i, row in enumerate(work_df.index):
        model_name = maier_to_long_name[row]

        new_dict[model_name] = {'Values': [], 'std': []}
        for key in timepoints:
            
            values = work_df.loc[row, stage_dict[key]].tolist()
            stds = [work_df.loc[row, stage_dict[key]].std()] * len(values)
            
            all_vals.extend(values)
            index.extend([inde]*len(values))
            inde+=1
            
            new_dict[model_name]['Values'].append(values)
            new_dict[model_name]['std'].append(stds)
            
        #new_dict[model_name]['Values'] = np.asarray(new_dict[model_name]['Values'])
        #new_dict[model_name]['std'] = np.asarray(new_dict[model_name]['std'])
        names.append(model_name)
        
    metas_dict, net = metabolomics_test_dict(model, metabolites, timepoints)
    new_dict.update(metas_dict)
    
    new_df = pd.DataFrame.from_dict(new_dict)
    print(all_vals+ net)
    return new_df, all_vals+net

In [94]:
mod = te.loada("model_files/SS_PLModel_MM.atm")
df= mke_test_dict(mod)
#names = df.index.tolist()
df

[1341.125, 1464.8750000000002, 2191.375, 56842.50000000001, 49832.5, 49652.5, 25268.41448189764, 27801.87265917603, 32662.92134831461, 56195.91078066915, 91018.5873605948, 43435.68773234201, 416.50000000000006, 356.87500000000006, 894.0, 37777.50000000001, 27087.5, 25367.5, 11422.72159800251, 13002.247191011238, 17082.771535580527, 21999.25650557621, 36286.98884758364, 14434.200743494424, 267.25, 278.875, 412.50000000000006, 9837.5, 6407.5, 4820.000000000001, 3022.2222222222213, 5416.85393258427, 4652.434456928839, 4741.263940520446, 7476.579925650559, 2619.330855018587, 1.500712181958364, nan, 1200.3147564215446, nan, 261.53490307988085, nan, 314.69392059549307, nan, 57.081030434100974, nan, 11567.80024323354, nan, 2.6153490307988085, nan, 752.5289405544399, nan, 9.382045729532232, nan, 10075.180857009856, nan, 184.94253860648715, 506.4644154880232, nan, 572.7926894633381, 1293.6034680605248, nan, 160.03445259887948, 37.56969639480828, nan, 1952.8436969759848, 1243.8494885197356, nan,

/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



(                                  Phosphatidylcholine_mem  \
 Values  [[1341.125, 1464.8750000000002, 2191.375], [56...   
 std     [[459.35486917342377, 459.35486917342377, 459....   
 
                              Phosphatidylethanolamine_mem  \
 Values  [[416.50000000000006, 356.87500000000006, 894....   
 std     [[294.4103305054585, 294.4103305054585, 294.41...   
 
                                    Phosphatidylserine_mem  \
 Values  [[267.25, 278.875, 412.50000000000006], [9837....   
 std     [[80.71383994293258, 80.71383994293258, 80.713...   
 
                                          CDP_ethanolamine  \
 Values  [[1.500712181958364], [nan], [1200.31475642154...   
 std     [[0.3756969639480828], [nan], [104.48335703565...   
 
                                              Ethanolamine  \
 Values  [[261.53490307988085], [nan], [314.69392059549...   
 std     [[51.47671108238925], [nan], [39.3056438372236...   
 
                                         Choline_phosphate  

In [91]:
len(t)
len(dic_dff.values.flatten('F'))

56

In [106]:
3*((1341-7200)**2)

102983643

In [108]:
residuals_timepoint

array([9.23043994e+07, 1.66357462e+09, 3.06447357e+10, 3.70198384e+10,
       1.09904429e+06, 2.80432680e+09, 5.91332593e+08, 2.00901434e+09,
       3.17435828e+05, 1.61022736e+08, 6.00951000e+07, 8.52100539e+07,
       7.16502958e-01,            nan, 1.43918534e+06,            nan,
       5.97130433e+27,            nan, 1.93470260e+30,            nan,
       3.80680309e+02,            nan, 1.32946209e+08,            nan,
       4.66031890e+27,            nan, 1.50994332e+30,            nan,
       7.02586906e+01,            nan, 1.01489120e+08,            nan,
       5.75991336e+14,            nan, 1.03674575e+16,            nan,
       2.66292998e+04,            nan, 5.35436867e+06,            nan,
                  nan,            nan, 6.93411762e+07,            nan,
                  nan,            nan, 1.72465507e+07,            nan])

In [110]:
residuals_all = (np.asarray(t) - dic_dff.values.flatten('F')[index])**2
residuals_timepoint = np.bincount(index, residuals_all)
real_res = np.nansum(residuals_timepoint.reshape(4,12, order='F'), axis=0)
residuals_timepoint.reshape(12,4)

array([[9.23043994e+07, 1.66357462e+09, 3.06447357e+10, 3.70198384e+10],
       [1.09904429e+06, 2.80432680e+09, 5.91332593e+08, 2.00901434e+09],
       [3.17435828e+05, 1.61022736e+08, 6.00951000e+07, 8.52100539e+07],
       [7.16502958e-01,            nan, 1.43918534e+06,            nan],
       [5.97130433e+27,            nan, 1.93470260e+30,            nan],
       [3.80680309e+02,            nan, 1.32946209e+08,            nan],
       [4.66031890e+27,            nan, 1.50994332e+30,            nan],
       [7.02586906e+01,            nan, 1.01489120e+08,            nan],
       [5.75991336e+14,            nan, 1.03674575e+16,            nan],
       [2.66292998e+04,            nan, 5.35436867e+06,            nan],
       [           nan,            nan, 6.93411762e+07,            nan],
       [           nan,            nan, 1.72465507e+07,            nan]])

In [431]:
it = 0
tester = dic_dff.values.flatten('F')
residuals2 = np.empty(len(tester))
for i in range(len(df.Values)):
    for ii in range(len(df.Values[i])):
        print(df.Values[i][ii])
        print(np.nansum(df.Values[i][ii]-tester[it]))
        residuals2[it] = np.nansum(df.Values[i][ii]-tester[it])
        it+=1
residuals2
residuals2.reshape(4,12, order='F')

[1341.125, 1464.8750000000002, 2191.375]
4994.375000176771
[56842.50000000001, 49832.5, 49652.5]
156324.5000000884
[25268.41448189764, 27801.87265917603, 32662.92134831461]
85730.20848942995
[56195.91078066915, 91018.5873605948, 43435.68773234201]
190647.18587364204
[416.50000000000006, 356.87500000000006, 894.0]
1665.6097323075815
[37777.50000000001, 27087.5, 25367.5]
90230.7347323076
[11422.72159800251, 13002.247191011238, 17082.771535580527]
41505.97505690185
[21999.25650557621, 36286.98884758364, 14434.200743494424]
72718.68082896186
[267.25, 278.875, 412.50000000000006]
957.7037814113402
[9837.5, 6407.5, 4820.000000000001]
21064.078781411343
[3022.2222222222213, 5416.85393258427, 4652.434456928839]
13090.58939314667
[4741.263940520446, 7476.579925650559, 2619.330855018587]
14836.253502600934
[2.61534903]
-16970778.197528336
[nan]
0.0
[752.52894055]
-71998296.80392247
[nan]
0.0
[261.53490308]
-77274215140345.9
[nan]
0.0
[314.6939206]
-1390935872510227.5
[nan]
0.0
[57.08103043]
-682

array([[ 4.99437500e+03,  1.66560973e+03,  9.57703781e+02,
        -1.69707782e+07, -7.72742151e+13, -6.82665284e+13,
         5.00712182e-01,  8.38204573e+00,  6.90098460e+02,
         1.22464149e+02,  0.00000000e+00,  0.00000000e+00],
       [ 1.56324500e+05,  9.02307347e+04,  2.10640788e+04,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.57302085e+04,  4.15059751e+04,  1.30905894e+04,
        -7.19982968e+07, -1.39093587e+15, -1.22879751e+15,
         1.19931476e+03,  1.00741809e+04,  1.86508766e+03,
         3.12155319e+03,  9.60673730e+03,  5.34115671e+03],
       [ 1.90647186e+05,  7.27186808e+04,  1.48362535e+04,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [309]:
residuals.reshape(4,12, order='F')

array([[ 4.99437500e+03,  1.66560973e+03,  9.57703781e+02,
        -1.69707782e+07, -7.72742151e+13, -6.82665284e+13,
         5.00712182e-01,  8.38204573e+00,  6.90098460e+02,
         1.22464149e+02,  0.00000000e+00,  0.00000000e+00],
       [ 1.56324500e+05,  9.02307347e+04,  2.10640788e+04,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.57302085e+04,  4.15059751e+04,  1.30905894e+04,
        -7.19982968e+07, -1.39093587e+15, -1.22879751e+15,
         1.19931476e+03,  1.00741809e+04,  1.86508766e+03,
         3.12155319e+03,  9.60673730e+03,  5.34115671e+03],
       [ 1.90647186e+05,  7.27186808e+04,  1.48362535e+04,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [71]:
res = mod.simulate(0, 50*3600)

In [72]:
dff = pd.DataFrame(res, columns=['time']
                            + mod.getFloatingSpeciesIds())
dff.columns

Index(['time', 'Phosphatidylserine', 'Ethanolamine_phosphate', 'L_Serine',
       'Ethanolamine', 'Choline', 'Choline_phosphate', 'CDP_ethanolamine',
       'CDP_choline', 'Phosphatidylcholine', 'Phosphatidylethanolamine',
       'Phosphatidylcholine_mem', 'Phosphatidylethanolamine_mem',
       'Phosphatidylserine_mem'],
      dtype='object')

In [94]:
dff[names]

,Phosphatidylcholine_mem,Phosphatidylethanolamine_mem,Phosphatidylserine_mem,CDP_choline,Ethanolamine,Choline_phosphate,CDP_ethanolamine,Ethanolamine_phosphate,L_Serine,Choline,Phosphatidylcholine,Phosphatidylethanolamine
0,0.999998,0.588423,0.307073,6.324483e+05,1.073253e+11,9.481464e+10,1.0,1.0,0.654247,37.57,0.999998,0.588423
1,1.000000,0.588423,0.307073,7.615862e+06,1.556217e+13,1.374812e+13,1.0,1.0,0.654247,37.57,1.000000,0.588423
2,1.000000,0.588423,0.307073,1.075182e+07,3.101701e+13,2.740143e+13,1.0,1.0,0.654247,37.57,1.000000,0.588423
3,1.000000,0.588423,0.307073,1.316057e+07,4.647185e+13,4.105473e+13,1.0,1.0,0.654247,37.57,1.000000,0.588423
4,1.000000,0.588423,0.307073,1.519208e+07,6.192670e+13,5.470804e+13,1.0,1.0,0.654247,37.57,1.000000,0.588423
5,1.000000,0.588423,0.307073,1.698231e+07,7.738154e+13,6.836134e+13,1.0,1.0,0.654247,37.57,1.000000,0.588423
6,1.000000,0.588423,0.307073,1.860099e+07,9.283638e+13,8.201465e+13,1.0,1.0,0.654247,37.57,1.000000,0.588423
7,1.000000,0.588423,0.307073,2.008978e+07,1.082912e+14,9.566795e+13,1.0,1.0,0.654247,37.57,1.000000,0.588423
8,1.000000,0.588423,0.307073,2.147554e+07,1.237461e+14,1.093213e+14,1.0,1.0,0.654247,37.57,1.000000,0.588423
9,1.000000,0.588423,0.307073,2.277715e+07,1.392009e+14,1.229746e+14,1.0,1.0,0.654247,37.57,1.000000,0.588423


In [75]:
def simulation_to_dict(simulation_result_p, timepoints=["uninfected", "ring", "troph", "schizont"]):
        # TODO : all time points are used but what if looking at specific time
        s_panda = simulation_result_p

        times = {'uninfected': 2 * 3600,
                 'ring': 8 * 3600,
                 'troph': 36 * 3600,
                 'schizont': 48 * 3600}

        # find most similar timepoint in simulation_results
        can = []
        for item in timepoints:
            can.append(min(s_panda['time'], key=lambda x: abs(x - times[item])))
        print(can)
        # cut df to size of the matching time points
        s_panda = s_panda[s_panda['time'].isin(can)]
    
        # dict with molecule names as keys and value is list of values at the specified timepoints
        return s_panda

In [84]:
dic_dff = simulation_to_dict(dff)
dic_dff.values.flatten('F')

[7200.0, 28800.0, 129600.0, 172800.0]


array([7.20000000e+03, 2.88000000e+04, 1.29600000e+05, 1.72800000e+05,
       3.07072863e-01, 3.07072863e-01, 3.07072863e-01, 3.07072863e-01,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       6.54247205e-01, 6.54247205e-01, 6.54247205e-01, 6.54247205e-01,
       7.72742151e+13, 3.09096861e+14, 1.39093587e+15, 1.85458116e+15,
       3.75700000e+01, 3.75700000e+01, 3.75700000e+01, 3.75700000e+01,
       6.82665284e+13, 2.73066114e+14, 1.22879751e+15, 1.63839668e+15,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.69707808e+07, 3.39409941e+07, 7.19990493e+07, 8.31372075e+07,
       9.99999941e-01, 9.99999971e-01, 9.99999986e-01, 9.99999988e-01,
       5.88422564e-01, 5.88422564e-01, 5.88422564e-01, 5.88422564e-01,
       9.99999941e-01, 9.99999971e-01, 9.99999986e-01, 9.99999988e-01,
       5.88422564e-01, 5.88422564e-01, 5.88422564e-01, 5.88422564e-01,
       3.07072863e-01, 3.07072863e-01, 3.07072863e-01, 3.07072863e-01])

In [257]:
ar = dic_dff.T.iloc[0].to_numpy()
ar

array([0.99999994, 0.99999997, 0.99999999, 0.99999999])

In [262]:
dic_dff.T

,2,8,36,48
Phosphatidylcholine_mem,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00
Phosphatidylethanolamine_mem,5.884226e-01,5.884226e-01,5.884226e-01,5.884226e-01
Phosphatidylserine_mem,3.070729e-01,3.070729e-01,3.070729e-01,3.070729e-01
CDP_choline,1.697078e+07,3.394099e+07,7.199905e+07,8.313721e+07
Ethanolamine,7.727422e+13,3.090969e+14,1.390936e+15,1.854581e+15
Choline_phosphate,6.826653e+13,2.730661e+14,1.228798e+15,1.638397e+15
CDP_ethanolamine,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
Ethanolamine_phosphate,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
L_Serine,6.542472e-01,6.542472e-01,6.542472e-01,6.542472e-01
Choline,3.757000e+01,3.757000e+01,3.757000e+01,3.757000e+01


In [260]:
np.asarray(df['Values'][0]).T-dic_dff.T.iloc[0].to_numpy()

array([[ 1340.12500006, 56841.50000003, 25267.41448191, 56194.91078068],
       [ 1463.87500006, 49831.50000003, 27800.87265919, 91017.58736061],
       [ 2190.37500006, 49651.50000003, 32661.92134833, 43434.68773235]])

In [272]:
np.asarray(df.values[0])

array([list([[1341.125, 1464.8750000000002, 2191.375], [56842.50000000001, 49832.5, 49652.5], [25268.41448189764, 27801.87265917603, 32662.92134831461], [56195.91078066915, 91018.5873605948, 43435.68773234201]]),
       list([[459.35486917342377, 459.35486917342377, 459.35486917342377], [4100.174793022045, 4100.174793022045, 4100.174793022045], [3757.8126725459665, 3757.8126725459665, 3757.8126725459665], [24629.164397334378, 24629.164397334378, 24629.164397334378]])],
      dtype=object)

In [267]:
for x in range(12):
    print(x)
    np.asarray(df['Values'][x]).T-dic_dff.T.iloc[x].to_numpy()

0
1
2
3
4
5
6
7
8


TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [191]:
df

,Values,std
Phosphatidylcholine_mem,"[[1341.125, 1464.8750000000002, 2191.375], [56...","[[459.35486917342377, 459.35486917342377, 459...."
Phosphatidylethanolamine_mem,"[[416.50000000000006, 356.87500000000006, 894....","[[294.4103305054585, 294.4103305054585, 294.41..."
Phosphatidylserine_mem,"[[267.25, 278.875, 412.50000000000006], [9837....","[[80.71383994293258, 80.71383994293258, 80.713..."
CDP_choline,"[[2.6153490307988085], [nan], [752.52894055443...","[[0.0026153490307988], [nan], [55.102532341424..."
Ethanolamine,"[[261.53490307988085], [nan], [314.69392059549...","[[51.47671108238925], [nan], [39.3056438372236..."
Choline_phosphate,"[[57.081030434100974], [nan], [11567.800243233...","[[14.986365081243967], [nan], [611.35202360745..."
CDP_ethanolamine,"[[1.500712181958364], [nan], [1200.31475642154...","[[0.3756969639480828], [nan], [104.48335703565..."
Ethanolamine_phosphate,"[[9.382045729532232], [nan], [10075.1808570098...","[[2.241727740684693], [nan], [945.325611274998..."
L_Serine,"[[184.94253860648715, 506.4644154880232], [nan...","[[48.77833509823175, 105.23666338214254], [nan..."
Choline,"[[160.03445259887948, 37.56969639480828], [nan...","[[24.9080860076077, 12.495556480483195], [nan]..."


In [63]:
df.index.tolist()

['Phosphatidylcholine_mem',
 'Phosphatidylethanolamine_mem',
 'Phosphatidylserine_mem',
 'CDP_choline',
 'Ethanolamine',
 'Choline_phosphate',
 'CDP_ethanolamine',
 'Ethanolamine_phosphate',
 'L_Serine',
 'Choline',
 'Phosphatidylcholine',
 'Phosphatidylethanolamine']

In [64]:
import numpy as np
a1 = [3.57841852e+02, 1.08442736e+03, 2.40627375e+03, 6.30087421e+08,
 1.29514403e+03, 2.24420784e+12, 1.75095167e+03, 1.73529899e+05,
 3.49618811e+03, 1.65810006e+22, 2.47701858e+28, 4.08921529e+25]
b=[1.03685643e+03, 1.89026214e+08, 3.13328207e+02, 1.29514403e+02,
 1.75095167e+02, 1.73529899e+04, 1.65810006e+21, 2.47701858e+27,
 8.14973281e+24, 3.21794700e+02, 7.15485712e+01, 3.71130477e+11]


print(sorted(a1))
print(sorted(b))
np.asarray(sorted(a1))==np.asarray(sorted(b))*10

[357.841852, 1084.42736, 1295.14403, 1750.95167, 2406.27375, 3496.18811, 173529.899, 630087421.0, 2244207840000.0, 1.65810006e+22, 4.08921529e+25, 2.47701858e+28]
[71.5485712, 129.514403, 175.095167, 313.328207, 321.7947, 1036.85643, 17352.9899, 189026214.0, 371130477000.0, 1.65810006e+21, 8.14973281e+24, 2.47701858e+27]


array([False, False, False, False, False, False,  True, False, False,
       False, False,  True])

In [66]:
names = np.array(['Phosphatidylcholine_mem',
 'Phosphatidylethanolamine_mem',
 'Phosphatidylserine_mem',
 'CDP_choline',
 'Ethanolamine',
 'Choline_phosphate',
 'CDP_ethanolamine',
 'Ethanolamine_phosphate',
 'L_Serine',
 'Choline',
 'Phosphatidylcholine',
 'Phosphatidylethanolamine'])
alex_bias_lst = np.array(['DAG', 'Phosphatidylserine_mem',
                         'Phosphatidylethanolamine_mem', 
                         'Phosphatidylcholine_mem'])
bias = np.in1d(names, alex_bias_lst)
b = np.asarray(b)
b[bias]= b[bias]*2
b.sort()
b

array([1.43097142e+02, 2.59028806e+02, 3.21794700e+02, 3.50190334e+02,
       6.26656414e+02, 2.07371286e+03, 1.73529899e+04, 3.78052428e+08,
       3.71130477e+11, 1.65810006e+21, 8.14973281e+24, 2.47701858e+27])